# Sistema de visão computacional

In [47]:
from skimage.io import imread, imshow, imsave
from skimage.color import rgb2gray
from skimage import filters
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
from skimage.exposure import histogram, equalize_hist
from skimage.filters import threshold_otsu, threshold_niblack
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from skimage.feature  import graycomatrix, graycoprops, local_binary_pattern
import warnings
warnings.filterwarnings("ignore")
from skimage.transform import resize






In [66]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,cohen_kappa_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

## 1. Pré-processamento

In [3]:
def crop_image(image):
    
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = rgb2gray(image)

    # Binariza a imagem
    thresh_mean = filters.threshold_mean(gray)
    binary_image = np.where(gray > thresh_mean, 255, 0).astype(np.uint8)
    
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    longest_contour = contours[0]
    longest_length = len(longest_contour)
    
    # Percorrer os contornos e encontrar o mais longo
    for contour in contours:
        length = len(contour)
        if length > longest_length:
            longest_contour = contour
            longest_length = length
            
    x, y, w, h = cv2.boundingRect(longest_contour)
    image = image[y:y+h, x:x+w]

    return image

In [4]:
# lista os arquivos do diretorio.
#tumor_paths = glob.glob('imagens/*.jpg')
#no_tumor_paths = glob.glob('imagens/*.jpg')


In [5]:
# Ler as imagens e faz o corte.
#imgs_tumor = [imread(i) for i in tumor_paths]
#imgs_tumor_crop = [crop_image(img) for img in imgs_tumor]


In [6]:
#imgs_no_tumor = [imread(i) for i in no_tumor_paths]
#imgs_no_tumor_crop = [crop_image(img) for img in imgs_no_tumor]


In [7]:
def save_crop(imgs, path):
    
    for i in range(len(imgs)):
        imsave(path+str(i)+'.jpg', imgs[i])
    
    return 1


### Transformações de Intensidade

In [8]:
def Alargamento_de_contraste(image, k, E):
  return 1/(1 + (k/image)**E)

In [9]:
def negativo(image):
  return 255 - image

In [10]:
def logaritmica(image, c):
  return c * np.log(1 + image)

In [11]:
def potencia(image, gamma):
  return (image / 255) ** gamma * 255

In [12]:
def intensity_transformation(imgs, type):
    
    if type == 'al':
        imgs_it = [Alargamento_de_contraste(img, 5, 2) for img in imgs]
    
    elif type == 'neg':
        imgs_it = [negativo(img) for img in imgs]
    
    elif type == 'log':
        imgs_it = [logaritmica(img, 0.5) for img in imgs]
    
    elif type == 'po':
        imgs_it = [potencia(img, 3) for img in imgs]
    
    return imgs_it


### Carregando as imagens pré-processadas

In [13]:
def dataloader( no_tumor, tumor):
 
    y = np.concatenate((np.zeros(len(no_tumor)), np.ones(len(tumor))), axis=0)

    x = no_tumor + tumor

    x = [resize(imread(img),(340,360)) for img in x]
    x = [(rgb2gray(img) * 255).astype(np.uint8) for img in x]
    
    return x,y



In [14]:
test_0 = glob.glob('dataset/testing/no_tumor/*.jpg')
test_1 = glob.glob('dataset/testing/tumor/*.jpg')
x_test, y_test = dataloader(test_0,test_1)

train_0 = glob.glob('dataset/training/no_tumor/*.jpg')
train_1 = glob.glob('dataset/training/tumor/*.jpg')
x_train, y_train = dataloader(train_0,train_1)



## 2. Segmentação

In [15]:
def kmeans_seg(img, k, cen='min'):
    kmeans = KMeans(n_clusters=k)
    img_train = img.reshape(-1, 1)
    result = kmeans.fit(img_train)
    
    labels = result.labels_
    cluster_centers = result.cluster_centers_
    
    cluster_centers = cluster_centers.reshape(1, -1)
    cluster_centers = list(cluster_centers[0])
    
    index_min = cluster_centers.index(min(cluster_centers))
    index_max = cluster_centers.index(max(cluster_centers))

    if cen == 'min':
        labels = (labels != index_min)
    elif cen == 'max':
        labels = (labels != index_max)
    
    img_final = labels.reshape(img.shape)
    
    return img_final.astype(np.uint8)


In [16]:
def segmentation(images):
    
    imgs_blur = [filters.gaussian(img, sigma=4) for img in images]
    k_seg = [kmeans_seg(img, 3) for img in imgs_blur]
    imgs_seg = [images[i] * k_seg[i] for i in range(len(images))]

    return imgs_seg


In [17]:
x_train_seg = segmentation(x_train)
X_test_seg = segmentation(x_test)


## Descritores de características

### GLCM

In [48]:
def glcm(images, distance):
    
    features = np.zeros((len(images), 6))

    for i, img in enumerate(images):
        matrix0 = graycomatrix(img, [distance], [0], normed=True)
        matrix1 = graycomatrix(img, [distance], [np.pi/4], normed=True)
        matrix2 = graycomatrix(img, [distance], [np.pi/2], normed=True)
        matrix3 = graycomatrix(img, [distance], [3*np.pi/4], normed=True)
        matrix = (matrix0+matrix1+matrix2+matrix3)/4

        props = np.zeros(6)
        props[0] = graycoprops(matrix,'contrast')
        props[1] = graycoprops(matrix,'dissimilarity')
        props[2] = graycoprops(matrix,'homogeneity')
        props[3] = graycoprops(matrix,'energy')
        props[4] = graycoprops(matrix,'correlation')
        props[5] = graycoprops(matrix,'ASM')
        features[i] = props
    
    return features



### LBP

In [56]:
#criando histograma, para tranformar saida do lbp em uma matriz unidimensional
def create_histograms(image, sub_images_num, bins_per_sub_images):

    grid = np.arange(0, image.shape[1]+1, image.shape[1]//sub_images_num)

    sub_image_histograms = []

    for i in range(1, len(grid)):
        for j in range(1, len(grid)):
            sub_image = image[grid[i-1]:grid[i], grid[j-1]:grid[j]]

            sub_image_histogram = np.histogram(sub_image, bins=bins_per_sub_images)[0]
            sub_image_histograms.append(sub_image_histogram)

    histogram = np.array(sub_image_histograms).flatten()
        
        
    return histogram

In [57]:
#lbp

def extract_lbp(images, n_points):
    
    ft = []
    for i, img in enumerate(images):
        lpb = local_binary_pattern(img, n_points,1,method='default')
        ft.append(create_histograms(lpb, 3, 64))
    
   
    return np.array(ft)
        

In [58]:
def extract_features(images, points, distance):
    
    x_glcm = glcm(images, distance) 
    
    x_lbp = extract_lbp(images,points)
    
    return x_glcm, x_lbp


In [59]:
x_train_glcm, x_train_lbp = extract_features(x_train, 8, 5)
x_test_glcm,x_test_lbp  = extract_features(x_test, 8, 5)

In [60]:
x_train_glcm_ns, x_train_lbp_ns = extract_features(x_train, 8, 5)
x_test_glcm_ns,  x_test_lbp_ns = extract_features(x_test, 8, 5)

# classificação

In [67]:
def cross_validation(model,x,y, test):
    acc_rf = []
    kappa = []
    for i in range(100):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test)
        model.fit(X_train,y_train)
        pred = model.predict(X_test)
        acc_rf.append(accuracy_score(y_test, pred))
        kappa.append(cohen_kappa_score(y_test,pred))
        
    return np.mean(acc_rf),np.std(acc_rf), np.mean(kappa)
    

def classfication(x_train, y_train, x_test, y_test):
    
    x = np.append(x_train,x_test,0)
    y = np.append(y_train,y_test,0)
    c_rf = RandomForestClassifier()
    
    print("Random Forest: Acc = {} Std = {}, kappa = {}".format(*cross_validation(c_rf,x,y,0.1)))
    
    
    mlp = MLPClassifier()
    print(" MLP : Acc = {} Std = {}, kappa = {}".format(*cross_validation(mlp,x,y,0.1)))
    
    model_svm = SVC()
    print("SVC : Acc = {} Std = {}, kappa = {}".format(*cross_validation(model_svm,x,y,0.1)))
    
    
    return True


### Com segmentação

In [68]:
classfication(x_train_glcm, y_train, x_test_glcm, y_test)


Random Forest: Acc = 0.9102597402597402 Std = 0.03301333442141993, kappa = 0.8181828015211714
 MLP : Acc = 0.5428571428571428 Std = 0.080498530999022, kappa = 0.08712063087355199
SVC : Acc = 0.6645454545454546 Std = 0.05758550514759241, kappa = 0.3123016527845898


True

In [69]:
classfication(x_train_lbp, y_train, x_test_lbp, y_test)

Random Forest: Acc = 0.9149350649350648 Std = 0.031243147334336134, kappa = 0.8283126872980879
 MLP : Acc = 0.8374025974025975 Std = 0.0669676681554246, kappa = 0.6713929768455703
SVC : Acc = 0.8316883116883118 Std = 0.047359703061889045, kappa = 0.6624330737998048


True

### Sem segmentação

In [70]:
classfication(x_train_glcm_ns, y_train, x_test_glcm_ns, y_test)

Random Forest: Acc = 0.9124675324675324 Std = 0.0323175757779875, kappa = 0.8226047302144112
 MLP : Acc = 0.5451948051948051 Std = 0.08309618248714307, kappa = 0.08408726482769174
SVC : Acc = 0.6737662337662339 Std = 0.055716556124840944, kappa = 0.33523197092328544


True

In [71]:
classfication(x_train_lbp_ns, y_train, x_test_lbp_ns, y_test)

Random Forest: Acc = 0.9197402597402597 Std = 0.03214803841225954, kappa = 0.8381764160862187


In [ ]:
#plt.scatter(features_test_0[:,0],features_test_0[:,3], c=labels_test_0)